In [1]:
# Libraries needed to get the html from a site and to parse the html
from bs4 import BeautifulSoup as bs
import requests

In [2]:
# Saving the url to a variable, getting the url and parsing the html
mainURL = "https://www.sbs.com.au/food/cuisine/maltese"
req = requests.get(mainURL)
soup = bs(req.text, "html.parser")

In [16]:
# Base link that hrefs will be added onto
baseLink = 'https://www.sbs.com.au'

In [29]:
# The tag that stores the href to the recipes
hrefs = soup.findAll(class_='link-underlay')

In [30]:
# Simply printing out the contents to see what the hrefs of the tags are
for i in hrefs:
    print(i.get('href'), "\n")

/food/recipes/maltese-baked-ricotta-pie 

/food/recipes/maltese-style-bruschetta-hobz-biz-zejt 

/food/recipes/maltese-rabbit-stew-stuffat-tal-fenek 

/food/recipes/bread-oil-hobz-biz-zejt 

/food/recipes/maltese-easter-biscuits-figolli 

/food/recipes/timpana 

/food/recipes/broad-bean-dip-bigilla 

/food/recipes/pears-baked-white-wine-almonds-and-armaretto-langas-mimli-bil-lewz 

/food/recipes/apple-and-fig-cake 

/food/recipes/pork-and-pea-pastizzi 

/food/recipes/rabbit-tomato-and-wine-sauce-rice 

/food/recipes/peach-and-almond-crumble 

/food/recipes/christmas-chocolate-salami-romblu-tal-milied-bic-cikkulata 

/food/recipes/pumpkin-and-rice-pie 

/food/recipes/grilled-garfish-roasted-potatoes-imsell-mixwi 

/food/article/2021/11/25/episode-guide-cook-adam-liaw-episodes-176-200 

https://www.sbs.com.au/ondemand/program/the-pizza-show 



In [37]:
# This for loop will generate the links to all the recipes that are shown on the mainURL
# However, it only shows those recipes that appear before clicking the
recipeLinks = []
for i in hrefs:
    fullLink = baseLink + i.get('href')
    if i.get('href')[6] == 'r':
        recipeLinks.append(fullLink)

In [38]:
# Printing the links to all the recipes shown before clicking the show more button
print(recipeLinks)

['https://www.sbs.com.au/food/recipes/maltese-baked-ricotta-pie', 'https://www.sbs.com.au/food/recipes/maltese-style-bruschetta-hobz-biz-zejt', 'https://www.sbs.com.au/food/recipes/maltese-rabbit-stew-stuffat-tal-fenek', 'https://www.sbs.com.au/food/recipes/bread-oil-hobz-biz-zejt', 'https://www.sbs.com.au/food/recipes/maltese-easter-biscuits-figolli', 'https://www.sbs.com.au/food/recipes/timpana', 'https://www.sbs.com.au/food/recipes/broad-bean-dip-bigilla', 'https://www.sbs.com.au/food/recipes/pears-baked-white-wine-almonds-and-armaretto-langas-mimli-bil-lewz', 'https://www.sbs.com.au/food/recipes/apple-and-fig-cake', 'https://www.sbs.com.au/food/recipes/pork-and-pea-pastizzi', 'https://www.sbs.com.au/food/recipes/rabbit-tomato-and-wine-sauce-rice', 'https://www.sbs.com.au/food/recipes/peach-and-almond-crumble', 'https://www.sbs.com.au/food/recipes/christmas-chocolate-salami-romblu-tal-milied-bic-cikkulata', 'https://www.sbs.com.au/food/recipes/pumpkin-and-rice-pie', 'https://www.sbs